# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
from crewai import Agent, Task, Crew
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

from utils import get_llm_api_key

GROQ_API_KEY = get_llm_api_key("GROQ")
GROQ_MODEL_NAME = "llama3-8b-8192"

groq_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name=GROQ_MODEL_NAME)

In [5]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | groq_llm
chain.invoke({"text": "Explain the importance of life."})

AIMessage(content="What a profound and intriguing topic! The importance of life is a question that has puzzled philosophers, scientists, and thinkers for centuries. In my humble opinion, the importance of life can be understood from various perspectives:\n\n1. **Meaning and Purpose**: Life provides us with the opportunity to find meaning and purpose. We strive to make sense of the world, and in doing so, we discover our individual purposes. This search for purpose gives our existence significance and direction.\n2. **Experiences and Memories**: Life is a tapestry of experiences, memories, and emotions. We form connections with others, learn from our mistakes, and cherish our triumphs. These experiences shape us, making our lives richer and more fulfilling.\n3. **Growth and Learning**: Life offers us the chance to learn, adapt, and evolve. We develop new skills, challenge ourselves, and expand our perspectives. This growth helps us become better versions of ourselves.\n4. **Relationship

## Role Playing, Focus and Cooperation

In [6]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
	llm=groq_llm
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [7]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	llm=groq_llm
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [8]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [17]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://claude.ai"
)

In [18]:
text = docs_scrape_tool.run()
print(text)

Using Tool: Read website content
ClaudeLoading...


##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [11]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
    max_iter=3
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [12]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
    max_iter=3
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [16]:
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory

memory = ConversationBufferMemory()
readonlymemory = ReadOnlySharedMemory(memory=memory)

crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=memory,
  max_rpm=29
)

ValidationError: 1 validation error for Crew
memory
  Input should be a valid boolean [type=bool_type, input_value=ConversationBufferMemory(), input_type=ConversationBufferMemory]
    For further information visit https://errors.pydantic.dev/2.7/v/bool_type

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [14]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [21]:
from crewai import Agent, Task, Crew
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from utils import get_llm_api_key

# Step 1: Retrieve Groq API Key
GROQ_API_KEY = get_llm_api_key("GROQ")
GROQ_MODEL_NAME = "llama3-70b-8192"

# Step 2: Initialize the Groq LLM
groq_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name=GROQ_MODEL_NAME)

# Step 3: Create a ConversationBufferMemory instance
memory = ConversationBufferMemory()

# Step 4: Initialize Agents with increased limits and shared memory
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working on providing "
        "support to {customer}, a super important customer for your company. "
        "You need to make sure that you provide the best support! "
        "Make sure to provide full complete answers, and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
    llm=groq_llm,
    memory=memory,  # Assign shared memory to the agent
    max_iterations=2,  # Increase the iteration limit
    time_limit=300  # Increase the time limit in seconds
)

support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the best support quality assurance in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working with your team "
        "on a request from {customer} ensuring that the support representative is "
        "providing the best support possible.\n"
        "You need to make sure that the support representative is providing full "
        "complete answers, and make no assumptions."
    ),
    verbose=True,
    llm=groq_llm,
    memory=memory,  # Assign shared memory to the agent
    max_iterations=2,  # Increase the iteration limit
    time_limit=300  # Increase the time limit in seconds
)

# Step 5: Define Tools
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

# Step 6: Define Tasks with increased limits and clear roles
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know to provide the best support possible. "
        "You must strive to provide a complete and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the customer's inquiry that addresses "
        "all aspects of their question. "
        "The response should include references to everything you used to find the answer, "
        "including external data or solutions. Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly tone throughout."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
    max_iterations=10,  # Increase the iteration limit
    time_limit=300  # Increase the time limit in seconds
)

quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
        "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to find the information, "
        "ensuring the response is well-supported and leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response ready to be sent to the customer. "
        "This response should fully address the customer's inquiry, incorporating all "
        "relevant feedback and improvements. "
        "Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
    max_iterations=10,  # Increase the iteration limit
    time_limit=300  # Increase the time limit in seconds
)

# Step 7: Initialize Crew without built-in memory
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=False  # Disable built-in memory
)

# Step 8: Define Inputs and Kickoff
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?"
}

# Step 9: Kickoff the Crew
result = crew.kickoff(inputs=inputs)

print(result)


2024-05-25 19:53:52,367 - 27552 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible. You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: I need to find the relevant information to help Andrew Ng from DeepLearningAI with setting up a Crew and kicking it off, specifically adding memory to his crew.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Activating Your CrewAI Team
 Initializing search
 crewAI
 crewAI
 crewAI
 Home

In [2]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew from DeepLearningAI,

Thank you for reaching out and trusting us with your important ask. I'm happy to assist you in setting up a Crew and kicking it off, specifically with adding memory to your crew.

To add memory to your crew, you need to define your agents with the 'memory=True' parameter when creating them. This will allow your agent to maintain a memory of past events and interactions, which can be useful for a variety of tasks. Here is an example of how to create an agent with memory:

from crewai import Agent
researcher = Agent(
role='Senior Researcher',
goal='Uncover groundbreaking technologies in {topic}',
verbose=True,
memory=True,
...)

Additionally, you can also add the 'cache=True' parameter when creating a crew. This will cache the results from tools to avoid hitting the rate limits and also improve the performance. Here is an example of how to create a crew with caching:

from crewai import Crew

crew = Crew(
...
cache=True
)

This will improve the performance and efficiency of your crew by caching the results from the tools.

I hope this information helps! Please let me know if you have any other questions or if there is anything else I can assist you with. I'm happy to help and I want to make sure you have the best experience with crewAI.

Best,
[Your name]

I have reviewed the response drafted by the Senior Support Representative and verified that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone. I have also checked for references and sources used to find the information, ensuring the response is well-supported and leaves no questions unanswered. The Senior Support Representative has confirmed that the answer is accurate and adheres to the high-quality standards expected for customer support and that the support representative has provided full, complete answers and makes no assumptions.

Confidence: 95%